# Example on how to use the code for fine-tuning your model

## Preliminary steps

### Paths and imports

Adds necessary repo folders to the path

In [1]:
import sys
sys.path.insert(1, '../preprocessing/') # preprocessing code
sys.path.insert(1, '../experiments/') # folder with configuration files
sys.path.insert(1, '../data/') # data code
sys.path.insert(1, '../model/') # model code

Imports

In [ ]:
from main_preprocessor import preprocess_data #for data preprocessing
import configparser # to automatically change the .ini file
from main_fine_tuner import FineTuner
#from sample import Sampler

(271914, 1)


## Model and data

Model settings

In [12]:
model = 'BIMODAL' # type of method to use
start = 'fixed' # position of the starting token ('fixed' or 'random')
aug_level = 1 # augmentation level (1 = no augmentation)
net_size = 512 # total number of hidden units

Fine tuning settings

In [13]:
fine_tuning = 'fine_tuning' #name and/or path to dataset for fine-tuning (csv or tar.xz)
epochs = 2 # fine tuning epochs

Sampling settings

In [ ]:
T_sampling = 0.7 # temperature for sampling
n_sampling = 10 # smiles to sample
n_folds = 1 # number of CV folds to use for fine-tuning
sample_valid=True
sample_novel=True
sample_unique=True

## Preprocessing the fine tuning library

In [ ]:
preprocess_data(filename_in='fine_tuning', model_type='BIMODAL', starting_point='fixed', augmentation=1)

In [15]:
fine_tuning_preprocessed = fine_tuning + '_' + model + '_' + start
print ('The new file can be found in ../data with the following name: "' + fine_tuning_preprocessed + '.csv".')

The new file can be found in ../data with the following name: "fine_tuning_BIMODAL_fixed.csv".


## Fine tuning the model

Create a the configuration file and saves it as (["Example_FineTuning.ini"](../experiments/)) based on the user-defined settings

In [17]:
# name of the configuration file to use
exp_name = 'Example_FineTuning'
template_name = 'BIMODAL_random_512_FineTuning_template.ini' # file to use as template

# reads the config file from the template
config = configparser.ConfigParser()
config.sections()
config.read('../experiments/' + template_name) # starts from one of the templates

# changes the fields based on the specified options
config['MODEL']['model'] = model
config['MODEL']['hidden_units'] = str(net_size//2)
config['DATA']['data'] = fine_tuning_preprocessed
config['TRAINING']['epochs'] = str(epochs) 

# writes back the new options
with open('../experiments/' + exp_name +'.ini', 'w') as configfile:
    config.write(configfile)

Fine tune the model

In [1]:
t = FineTuner(experiment_name = exp_name)
t.fine_tuning(stor_dir='../evaluation/', restart=False)

NameError: name 'FineTuner' is not defined

Sample from the fine-tuned model

In [ ]:
s = Sampler(ex_name)
s.sample(N=n_sampling, stor_dir='../evaluation', T=T_sampling, fold=n_folds, epoch=epochs, valid=sample_valid, novel=sample_novel, unique=sample_unique, write_csv=True)